# Projeto Final - (A -Tech)

## Clusterização

#### Para ajudar a A-Tech a definir uma estratégia de venda alinhada com as características de suas lojas e fatores externos vamos aplicar a técnica de clusterização com o objetivo de agrupar as lojas em grupos de tal forma que dentro de cada grupo as lojas sejam semelhantes e distintas entre os grupos.

In [ ]:
# Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as snb
import datetime
import statistics
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler

In [ ]:
### Importando os dados

In [ ]:
# Dataset lojas
lojas = pd.read_csv('lojas.csv',sep=',',decimal='.',header = 'infer')
lojas.head()

In [ ]:
lojas

In [ ]:
lojas.info()

In [ ]:
# Tansformando o tipo
lojas['loja'] = lojas['loja'].astype('str')
lojas['tipo_de_loja']= lojas['tipo_de_loja'].astype('str')
lojas.info()

In [ ]:
# Dataset outras_variaveis
outras_variaveis = pd.read_csv('outras_variaveis.csv',sep=',',decimal='.',header = 'infer')
outras_variaveis.head()

In [ ]:
outras_variaveis.info()

In [ ]:
# Tansformando o tipo
outras_variaveis['loja'] = outras_variaveis['loja'].astype('str')
outras_variaveis['data'] = pd.to_datetime(outras_variaveis.data,infer_datetime_format=True, format="%d/%m/%Y")

In [ ]:
print(outras_variaveis.data.max())
print(outras_variaveis.data.min())

In [ ]:
outras_variaveis.info()
outras_variaveis.head()

In [ ]:
# Dataset vendas
vendas = pd.read_csv('vendas.csv',sep=',',decimal='.',header = 'infer')
vendas.head()

In [ ]:
vendas.info()

In [ ]:
# Tansformando o tipo
vendas['loja'] = vendas['loja'].astype('str')
vendas['departamento'] = vendas['departamento'].astype('str')
vendas['data'] = pd.to_datetime(vendas.data,infer_datetime_format=True, format="%d/%m/%Y")
vendas['semana_de_evento'] = vendas['semana_de_evento'].astype('str')

In [ ]:
#Máxima e mínima data de venda
print(vendas.data.min())
print(vendas.data.max())

In [ ]:
vendas.info()
vendas.head()

In [ ]:
lojas.shape

In [ ]:
outras_variaveis.shape

In [ ]:
vendas.shape

In [ ]:
# Left Join - Vendas e lojas
df_aux = vendas.merge(lojas, on='loja', how='left')
df_aux['data'] = pd.to_datetime(df_aux.data)
vendas['loja'] = vendas['loja'].astype('str')
df_aux.head()

In [ ]:
df_aux.isnull().sum()

In [ ]:
df_aux.shape

In [ ]:
# Left Join - Vendas, lojas e outras variáveis
df = df_aux.merge(outras_variaveis, on=['loja','data'], how='left')
df.head()

In [ ]:
#Valores em branco em pc, IPC e td
df.isnull().sum()

In [ ]:
### Ajustando variáveis para o K-means (Feature Engineering)

In [ ]:
#Ajustando o df do K-means
df_cluster = df

# Drop departamento - Não vamos usar os departamentos na análise de cluster para lojas
df_cluster = df_cluster.drop(df.columns[1], axis=1)

# Drop evento - Já temos a variável semana de evento do dataset de vendas, vamos dropar a variável evento
df_cluster = df_cluster.drop(df.columns[10], axis=1)

df_cluster

In [ ]:
# Concatena as colunas criadas ao dataframe
#dummy semana_de_evento
df_cluster = pd.concat([df_cluster.drop("semana_de_evento", axis=1), pd.get_dummies(df_cluster.semana_de_evento)], axis=1)
df_cluster.head()

In [ ]:
# Concatena as colunas criadas ao dataframe
# dummy tipo de loja
df_cluster = pd.concat([df_cluster.drop("tipo_de_loja", axis=1), pd.get_dummies(df_cluster.tipo_de_loja)], axis=1)
df_cluster.head()

In [ ]:
# Data máxima e mínima
# Qtd de dias entre

print(df_cluster.data.max())
print(df_cluster.data.min())
qtd_dias_entre = (df_cluster.data.max() - df_cluster.data.min()).days
print(qtd_dias_entre)

In [ ]:
# Agrega tabela em nível cliente
aggregations = {
        'venda_semanal': ['sum','count'],
        'tamanho': 'max',
        'preco_combustivel':  'median',
        'IPC': 'median',
        'taxa_desemprego':  'median',
        'True':  'sum',
        'A': 'unique',
        'B': 'unique',
        'C': 'unique'
}

df_cluster_2 = df_cluster.groupby('loja', observed=True,as_index=False).agg(aggregations)
df_cluster_2

In [ ]:
df_cluster_2.info()

In [ ]:
# Nomeando as colunas
df_cluster_2 = df_cluster_2.set_axis(['loja', 'valor_venda','qtd_venda','tamanho','mediana_pc','mediana_ipc','mediana_td','qtd_eventos','A','B','C'], axis=1, inplace=False)

In [ ]:
# Calculando a proporção de eventos em relação a qtd de dias entre a data máx e mín
df_cluster_2['prop_evento'] = df_cluster_2['qtd_eventos']/qtd_dias_entre

# Excluindo a coluna qtd_eventos
df_cluster_2 = df_cluster_2.drop(df_cluster_2.columns[7], axis=1)

In [ ]:
df_cluster_2

In [ ]:
# Vamor criar a variável venda/tamanho
df_cluster_2['venda_tamanho'] = (df_cluster_2.valor_venda)/(df_cluster_2.tamanho)

In [ ]:
df_cluster_2  = df_cluster_2.reindex(columns=['loja', 'valor_venda','qtd_venda','tamanho','mediana_pc','mediana_ipc','mediana_td','prop_evento','venda_tamanho','A','B','C'])

In [ ]:
### Análise exploratória dos dados

In [ ]:
df_cluster_2

In [ ]:
df_cluster_2.info()

In [ ]:
# Correlação - agrupamento por loja
snb.set(rc = {'figure.figsize':(10,8)})
snb.heatmap(df_cluster_2.corr(), annot = True, fmt=".2f", linewidths=.6)

In [ ]:
# Dispersão - Loja e Tamanho

plt.figure(figsize=(10, 6)) 
snb.scatterplot(x='loja', 
                y='tamanho', 
                      sizes=(20, 500), 
                alpha=1, 
                hue='tipo_de_loja', 
                data=lojas) 
plt.legend(bbox_to_anchor=(1, 1), borderaxespad=0) 
plt.xlabel("Loja") 
plt.ylabel("Tamanho") 
plt.title("Tamanho por Loja") 
plt.tight_layout()

In [ ]:
#Valor de venda por loja
venda_loja = df.groupby('loja', as_index=False)['venda_semanal'].sum()
venda_loja = venda_loja.set_axis(['Loja', 'Valor de venda'], axis=1, inplace=False)
venda_loja = venda_loja.sort_values(by=['Valor de venda'],ascending=False)
fig = px.bar(venda_loja, x=venda_loja['Loja'], y=venda_loja['Valor de venda'], title="Valor de venda total por loja")
fig.update_layout(
    width=800,
    height=400,
    showlegend=False)
fig.show()

In [ ]:
# Venda por departamento
venda_dep = df.groupby('departamento', as_index=False)['venda_semanal'].sum()
venda_dep = venda_dep.set_axis(['Departamento', 'Valor de venda'], axis=1, inplace=False)
venda_dep = venda_dep.sort_values(by=['Valor de venda'],ascending=False)
fig = px.bar(venda_dep,x='Departamento',y='Valor de venda', title="Valor de venda total por departamento")
fig.update_layout(
    width=1000,
    height=600,
    showlegend=False)
fig.show()

In [ ]:
#Venda com incidência de evento
venda_dia = df.groupby(['data'], as_index=True).agg({'evento':  'mean',
                                                       'venda_semanal': 'sum'})
ax = venda_dia['venda_semanal'].plot()
venda_dia['venda_semanal'].loc[venda_dia['evento'] == 1].plot(ax=ax, kind='line',
                                                              marker='h',linestyle = 'None',
                                                              legend=True, label='Feriado', color='red')

plt.title('Valor de venda por semana')
plt.xlabel('Data')
plt.ylabel('Valor de venda')

In [ ]:
# Visualizando o comportamento das variáveis por loja ao longo do tempo
aggregations = {'evento':  'mean',
                'preco_combustivel': 'mean',
                'IPC': 'mean',
                'taxa_desemprego': 'mean',
                'venda_semanal': 'sum'}
df.reset_index(inplace=True)
plot_loja = df.groupby(['data','loja'], as_index=False).agg(aggregations)
plot_loja['loja'] = plot_loja['loja'].astype('int')
df.set_index('data', inplace=True)
plot_loja.set_index('data', inplace=True)

def plot_serie(loja):
    plt.rc("figure",figsize=(8,4))
    ax = plot_loja[['venda_semanal']].loc[(plot_loja['loja'] == loja)].plot(legend=False)
    plot_loja[['venda_semanal']].loc[(plot_loja['loja'] == loja) & (plot_loja['evento'] == 1)].plot(ax=ax,
                                                                                                    kind='line',
                                                                                                    marker='h',
                                                                                                    linestyle = 'None',
                                                                                                    legend=False, color='red')
    plt.title('Valor de venda semanal por loja')
    plt.xlabel('Data')
    plt.ylabel('Valor de venda')

    plot_loja[['preco_combustivel',
               'IPC',
               'taxa_desemprego'
              ]].loc[(plot_loja['loja'] == loja)].plot(subplots=True,
                                                       title='Valores de dados regionais por loja',
                                                       xlabel='Data')
    
    plt.show()
    
# Criando o Slider interativo
_ = interact(plot_serie,
             loja=IntSlider(min=plot_loja.loja.min(), max=plot_loja.loja.max(), step = 1, value = 1),
             )

In [ ]:
### Padronizando os dados

In [ ]:
# Padronizando as variáveis
escala=StandardScaler()
df_cluster_3 = pd.DataFrame(escala.fit_transform(df_cluster_2.iloc[:,1:9]),columns=['std_valor_venda','std_qtd_venda','std_tamanho','std_media_pc','std_media_ipc','std_media_td','std_prop_evento','std_venda_tamanho'])

In [ ]:
df_cluster_3

In [ ]:
#Adicionando as colunas A, B e C
df_cluster_4 = pd.concat([df_cluster_3, df_cluster_2], axis=1)
df_cluster_4.info()
df_cluster_4.head()

In [ ]:
# Dropando o que não está padronizado
df_cluster_4 = df_cluster_4.drop(df_cluster_4.columns[9:17], axis=1)

In [ ]:
df_cluster_4

In [ ]:
# Loja como índice
df_cluster_5 = df_cluster_4.set_index(['loja'])
df_cluster_5

In [ ]:
# Cálculo das iterações do k-means (Cotovelo)
# O número de iterações é subjetivo, utilizou-se 10, pois com esse número é possível avaliar
# o ponto de flexão da curva, mas se desejar é possível testar com valor maior ou menor

wcss = {}
for k in range(1, 15):
  print(k)
  kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=200, random_state=3)
  kmeans.fit(df_cluster_5)
  wcss[k] = kmeans.inertia_

# No código abaixo será gerado o gráfico que permite aplicação do método do "cotovelo"
fig = make_subplots(
    rows=1,
    cols=1,
    subplot_titles=['Método do Cotovelo'],
    shared_xaxes=True,
    shared_yaxes=False)


fig.add_trace(
    go.Scatter(
        x=list(wcss.keys()),
        y=list(wcss.values()),
        mode='lines+markers',
        textposition='top center',
        showlegend=False),
    row=1, 
    col=1)

fig.update_xaxes(visible=True, title='N° de clusters',row=1, col=1)
fig.update_yaxes(visible=True, title='Soma dos quadrados totais dos clusters',row=1, col=1)

fig.update_layout(
  title='Definição do Nº Ideal de Clusters',
  showlegend=False,
  xaxis_showticklabels=True,
  height=400,
  width=800,
  xaxis = dict(
    tickmode = 'linear',
    tick0 = 0,
    dtick = 1))

fig.show()

In [ ]:
# Definição do DF a ser Clusterizado
df_cluster_km = df_cluster_5.copy()

# Clusterização
clus = KMeans(n_clusters=5, init='k-means++', max_iter=300,random_state=3)
clus.fit(df_cluster_km)

# Ajustando as colunas
#df_cluster_km['CLUSTER'] = clus.labels_
df_cluster_km.loc[:, 'CLUSTER'] = clus.labels_
df_cluster_km.head()

In [ ]:
# Volumetria por Categoricas
df_cluster_km['CLUSTER'].value_counts().plot(kind="bar",color = ['b','r','m','g','orange'])
plt.title('Qtd de lojas por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Qtd de lojas')
plt.show()

In [ ]:
# Cluster
# Dataset não padronizado
df_cluster_2.loc[:, 'CLUSTER'] = clus.labels_
df_cluster_2.head()

In [ ]:
# Dispersão do valor de venda por cluster
snb.lmplot( x="CLUSTER", y="valor_venda", data=df_cluster_2, fit_reg=False, hue='CLUSTER', legend=True)
plt.title('Valor de venda por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Valor de venda')

In [ ]:
# Coordenadas Paralelas
fig = px.parallel_coordinates(df_cluster_2, color="CLUSTER",)
fig.update_layout(
    title={
        'text': "Cluster analysis",
        'y':1
        },
    height=500,
    width=1000)
fig.show()

In [ ]:
#analise_grupo_mean

In [ ]:
# Coordenadas Paralelas
# Variáveis utilizadas na clusterização

aggregations = {
        'valor_venda': 'mean',
        'qtd_venda': 'mean',
        'tamanho':  'mean',
        'mediana_pc': 'mean',
        'mediana_ipc': 'mean',
        'mediana_td':  'mean',
        'prop_evento':  'mean',
        'venda_tamanho': 'mean'
        
}

analise_grupo_mean = df_cluster_2.groupby(['CLUSTER']).agg(aggregations).reset_index()
analise_grupo_mean.head()

fig = px.parallel_coordinates(analise_grupo_mean, color='CLUSTER',labels={
                                             'valor_venda':'Venda',
                                             'qtd_venda':'Qtd de venda',
                                             'tamanho':'Tamanho',
                                             'mediana_pc':'Preço do combustível',
                                             'mediana_ipc': 'IPC',
                                             'mediana_td':'Taxa de desemprego',
                                             'prop_evento':'Prop. evento',
                                             'venda_tamanho':'Venda/tamanho'
                                             })
fig.update_layout(
    title={
        'text': "Análise de cluster - Média",
        'y':1
        },
    height=500,
    width=1000)
fig.show()

In [ ]:
# Distribuição de valor_venda por CLUSTER

plt.figure(figsize=(10, 5))

# plot a bar chart
snb.barplot(
    y="valor_venda", 
    x="CLUSTER", 
    data=df_cluster_2, 
    estimator=sum, 
    ci=None );

plt.title('Valor de venda por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Valor de venda')

In [ ]:
# Distribuição de qtd_venda por CLUSTER
# Set the figure size
plt.figure(figsize=(10, 5))

# plot a bar chart
snb.barplot(
    y="qtd_venda", 
    x="CLUSTER", 
    data=df_cluster_2, 
    estimator=sum, 
    ci=None );

plt.title('Qtd de venda por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Qtd de venda')

In [ ]:
# Distribuição de tamanho por CLUSTER
# Set the figure size
plt.figure(figsize=(10, 5))

# plot a bar chart
snb.barplot(
    y="tamanho", 
    x="CLUSTER", 
    data=df_cluster_2, 
    estimator=sum, 
    ci=None );

plt.title('Metragem (m²) por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Tamanho')

In [ ]:
plt.figure(figsize=(10, 5))
snb.boxplot(x= df_cluster_2['CLUSTER'], y=df_cluster_2['tamanho'], data=df_cluster_2).set(title='Tamanho total por Cluster', ylabel = 'Tamanho m²',xlabel = 'Cluster')

In [ ]:
# Distribuição da média de preço do combustível por CLUSTER
snb.boxplot(x= df_cluster_2['CLUSTER'], y=df_cluster_2['mediana_pc'], data=df_cluster_2).set(title='Preço médio do combustível por Cluster', ylabel = 'Preço médio do combustível',xlabel = 'Cluster')

In [ ]:
# Distribuição da média do IPC por CLUSTER
snb.boxplot(x= df_cluster_2['CLUSTER'], y=df_cluster_2['mediana_ipc'], data=df_cluster_2).set(title='IPC médio por Cluster', ylabel = 'IPC médio',xlabel = 'Cluster')

In [ ]:
# Distribuição da média da taxa de desemprego por CLUSTER
snb.boxplot(x= df_cluster_2['CLUSTER'], y=df_cluster_2['mediana_td'], data=df_cluster_2).set(title='Média da taxa de desemprego por Cluster', ylabel = 'Média da taxa de desemprego',xlabel = 'Cluster')

In [ ]:
# Distribuição da proporção de eventos por CLUSTER
snb.boxplot(x= df_cluster_2['CLUSTER'], y=df_cluster_2['prop_evento'], data=df_cluster_2).set(title='Proporção de eventos por Cluster', ylabel = 'Proporção de eventos ',xlabel = 'Cluster')

In [ ]:
# Distribuição da Proporção de venda/tamanho
snb.boxplot(x= df_cluster_2['CLUSTER'], y=df_cluster_2['venda_tamanho'], data=df_cluster_2).set(title='Proporção de venda/tamanho por Cluster', ylabel = 'Venda/tamanho ',xlabel = 'Cluster')

In [ ]:
df_cluster_2.to_csv("teste.csv", sep=";", decimal='.')

# Previsão de vendas 

###### Utilizaremos como técnica de previsão a série temporal (ARIMA) por ser o método que apresentou melhor resultado (MAE e MAPE).
###### As previsão serão feitas até o dia 26/04/2013, data  que representa o período de 6 meses.
###### Faremos previsões da venda semanal apenas por loja.
###### Utilizaremos a função auto_arima para encontrar o melhor modelo.

######  Entregas:
######   - Modelo de previsão de venda de todas as lojas;
######  - Modelo de previsão de venda para cada loja ( As lojas escolhidas foram as lojas do cluster 0 e 4 por representarem 44% da venda total de 05/05/2010 a 26/10/2012);
######  - Modelo de previsão da venda média de lojas do cluster 0; 
######   - Modelo de previsão da venda média de lojas do cluster 4.

In [ ]:
# importando bibliotecas utilizadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as snb
import datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima.arima import auto_arima

##### Modelo de previsão de venda de todas as lojas

In [ ]:
# carrega dados - Vendas
vendas = pd.read_csv('vendas.csv',sep=',',decimal='.',header = 'infer')
vendas.head()

# transforma dados
vendas['loja'] = vendas['loja'].astype('str')
vendas['departamento'] = vendas['departamento'].astype('str')
vendas['data'] = pd.to_datetime(vendas.data,infer_datetime_format=True, format="%d/%m/%Y")
vendas['semana_de_evento'] = vendas['semana_de_evento'].astype('str')

In [ ]:
vendas.info()
vendas.head()

In [ ]:
# Vamos usar apenas o dataset de vendas para fazer a previsão por série temporal
# Venda por dia de todas as lojas 

print(vendas.data.min())
print(vendas.data.max())

aggregations = {
        'venda_semanal': 'sum',
               }

venda_total = vendas.groupby('data').agg(aggregations)


venda_total.head()

In [ ]:
venda_total.info()

In [ ]:
# plot série temporal
venda_total['venda_semanal'].plot(title='Venda semanal')

# imprime plots
plt.show()

In [ ]:
venda_total

In [ ]:
# define frequência semanal
venda_semanal = venda_total[['venda_semanal']].asfreq('W-FRI')
venda_semanal.info()
venda_semanal.head()

In [ ]:
venda_semanal.isnull().sum()

In [ ]:
venda_semanal

In [ ]:
# plot série temporal
venda_semanal['venda_semanal'].plot(title='Venda semanal')

In [ ]:
# Train e test
train = venda_semanal[:'2012-06-29']
test = venda_semanal['2012-07-06':]

In [ ]:
test.shape
train.shape

In [ ]:
# Decomposição
decomposition = seasonal_decompose(train['venda_semanal'], model='multiplicative')  
# teste tambem com model='additive' e veja se você nota alguma diferença

# Outra forma possível de plotar a decomposição
plt.rc("figure",figsize=(8,4))
#plt.figsize=(20, 20)
decomposition.plot()
plt.show()

In [ ]:
# Buscando o melhor modelo pelo auto_arima
arima_model = auto_arima(train, start_p=1, start_q=2,
                           max_p=5, max_q=5, m=52,
                           start_P=1, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

arima_model.summary()

In [ ]:
#Fazendo a previsão
#n_periods = 17 até 26/10/2012
prediction = pd.DataFrame(arima_model.predict(n_periods = 17))
prediction.columns = ['predicted_sales']                          

#n_periods = 43 até 26/04/2013
prediction_6meses = pd.DataFrame(arima_model.predict(n_periods = 43))
prediction_6meses.columns = ['predicted_sales']    

In [ ]:
print(prediction.tail())
print(prediction_6meses.tail())

In [ ]:
# plot results
plt.plot(test['venda_semanal'], color='orange', label= 'teste')
plt.plot(prediction, color='red', label = 'previsão')
plt.legend()
plt.xticks(rotation = 45)
plt.show()

In [ ]:
# plot results - 6 meses
plt.plot(test['venda_semanal'], color='orange', label= 'teste')
plt.plot(prediction_6meses, color='red', label = 'previsão')
plt.legend()
plt.xticks(rotation = 45)
plt.show()

In [ ]:
# dataset para medir erro
modelo_forecast = pd.concat([train.append(test),prediction['predicted_sales'].rename('previsao')], axis=1, sort=False)
modelo_forecast.tail()

In [ ]:
# função reutilizável para cálculo de erros
def calculate_forecast_errors(df, prediction_size):
    
    df = df.copy()
    
    df['e'] = df['venda_semanal'] - df['previsao']
    df['p'] = 100 * df['e'] / df['venda_semanal']
    
    predicted_part = df[-prediction_size:]
    
    error_mean = lambda error_name: np.mean(np.abs(predicted_part[error_name]))
    
    return {'MAPE': error_mean('p'), 'MAE': error_mean('e')}

In [ ]:
# métricas de erro
for err_name, err_value in calculate_forecast_errors(modelo_forecast, len(test)).items():
    print(err_name, err_value)

#### Modelo de previsão de venda para cada loja ( As lojas escolhidas foram as lojas do cluster 0 e 4 - 19 lojas)

In [ ]:
# Tomando como base a análise de cluster foram selecionadas 19 lojas pertencentes aos clusters 0 e 4. Esses 2 clusters foram
# selecionados por representarem 63% da venda total no período de 05/02/2010 a 26/10/2012

# Modelo para loja individual
# Só é preciso modificar a variável cod_loja
# Não iremos fazer um modelo para cada uma das 19 lojas, mas vamos apresentar o códido utilizado para previsão de cada loja
# Um exmplo com a loja 1

# lojas selecionadas: 1,2,4,6,8,10,11,13,14,19,20,23,24,27,31,32,39,40,41

In [ ]:
# Venda semanal da loja 1 

cod_loja ='1'

vendas_L1 = vendas[(vendas['loja'] == cod_loja)]
                   
print(vendas_L1.data.min())
print(vendas_L1.data.max())

aggregations_L1 = {'venda_semanal': 'sum'}

venda_total_L1 = vendas_L1.groupby('data').agg(aggregations_L1)

venda_total_L1.head()

In [ ]:
# define frequência semanal
venda_semanal_L1 = venda_total_L1[['venda_semanal']].asfreq('W-FRI')
venda_semanal_L1.info()
venda_semanal_L1.head()

In [ ]:
# plot série temporal
venda_semanal_L1['venda_semanal'].plot(title='Venda semanal')

In [ ]:
# Train e test
train_L1 = venda_semanal_L1[:'2012-06-29']
test_L1 = venda_semanal_L1['2012-07-06':]

In [ ]:
# descomposição
decomposition_L1 = seasonal_decompose(train_L1['venda_semanal'], model='multiplicative')  
# teste tambem com model='additive' e veja se você nota alguma diferença

# Outra forma possível de plotar a decomposição
plt.figsize=(10, 7)
decomposition_L1.plot()
plt.show()

In [ ]:
#Buscando o modelo com a melhor performance
arima_model_L1 = auto_arima(train_L1, start_p=1, start_q=2,
                           max_p=5, max_q=5, m=52,
                           start_P=1, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
arima_model_L1.summary()

In [ ]:
#n_periods = 17 até 26/10/2012
prediction_L1 = pd.DataFrame(arima_model_L1.predict(n_periods = 17))
prediction_L1.columns = ['predicted_sales']                       

#n_periods = 43 até 26/04/2013
prediction_6meses_L1 = pd.DataFrame(arima_model_L1.predict(n_periods = 43))
prediction_6meses_L1.columns = ['predicted_sales']    

In [ ]:
print(prediction_L1.tail())
print(prediction_6meses_L1.tail())

In [ ]:
# plot results - L1
plt.plot(test_L1['venda_semanal'], color='orange', label= 'teste')
plt.plot(prediction_L1, color='red', label = 'previsão')
plt.legend()
plt.xticks(rotation = 45)
plt.show()

In [ ]:
#plot results - 6 meses
plt.plot(test_L1['venda_semanal'], color='orange', label= 'teste')
plt.plot(prediction_6meses_L1, color='red', label = 'previsão')
plt.legend()
plt.xticks(rotation = 45)
plt.show()

In [ ]:
# dataset para medir erro
modelo_forecast_L1 = pd.concat([train_L1.append(test_L1),prediction_L1['predicted_sales'].rename('previsao')], axis=1, sort=False)
modelo_forecast_L1.tail()

In [ ]:
# métricas de erro
for err_name, err_value in calculate_forecast_errors(modelo_forecast_L1, len(test_L1)).items():
    print(err_name, err_value)

#### Previsão do cluster 0

In [ ]:
# Venda semanal do cluster 0
# Iremos prever a venda média por semana das lojas do cluster 0

cluster_zero = ['10','13','14','19','20','23','24','27','4','40']

def listaFiltro(dataframe, valores):
    return dataframe.loc[dataframe['loja'].isin(valores)]


vendas_C0 = listaFiltro(vendas, cluster_zero)
vendas_C0


print(vendas_C0.data.min())
print(vendas_C0.data.max())

aggregations = {'venda_semanal': 'mean'}

venda_total_C0 = vendas_C0.groupby('data').agg(aggregations)


venda_total_C0.head()

In [ ]:
# define frequência semanal
venda_semanal_C0 = venda_total_C0[['venda_semanal']].asfreq('W-FRI')
venda_semanal_C0.info()
venda_semanal_C0.head()

In [ ]:
# plot série temporal
plt.rc("figure",figsize=(15,5))
venda_semanal_C0.plot(title='Venda semanal')

In [ ]:
venda_semanal_C0.isnull().sum()

In [ ]:
# Train e test
train_C0 = venda_semanal_C0[:'2012-06-29']
test_C0 = venda_semanal_C0['2012-07-06':]

In [ ]:
# Decomposição
decomposition_C0 = seasonal_decompose(train_C0, model='multiplicative')  
# teste tambem com model='additive' e veja se você nota alguma diferença

# Outra forma possível de plotar a decomposição
plt.rc("figure",figsize=(20,10))
decomposition_C0.plot()
plt.show()

In [ ]:
# Buscando o modelo com a melhor performance
arima_model_C0 = auto_arima(train_C0, start_p=1, start_q=2,
                           max_p=5, max_q=5, m=52,
                           start_P=1, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
arima_model_C0.summary()

In [ ]:
#n_periods = 17 até 26/10/2012
prediction_C0 = pd.DataFrame(arima_model_C0.predict(n_periods = 17))
prediction_C0.columns = ['predicted_sales']                       

#n_periods = 43 até 26/04/2013
prediction_6meses_C0 = pd.DataFrame(arima_model_C0.predict(n_periods = 43))
prediction_6meses_C0.columns = ['predicted_sales']   

In [ ]:
print(prediction_C0.tail())
print(prediction_6meses_C0.tail())

In [ ]:
# plot results - C0
plt.rc("figure",figsize=(15,5))
plt.plot(test_C0, color='orange', label= 'teste')
plt.plot(prediction_C0, color='red', label = 'previsão')
plt.legend()
plt.xticks(rotation = 45)
plt.show()

In [ ]:
#plot results - C0 - 6 meses
plt.plot(test_C0, color='orange', label= 'teste')
plt.plot(prediction_6meses_C0, color='red', label = 'previsão')
plt.legend()
plt.xticks(rotation = 45)
plt.show()

In [ ]:
# dataset para medir erro
modelo_forecast_C0 = pd.concat([train_C0.append(test_C0),prediction_C0['predicted_sales'].rename('previsao')], axis=1, sort=False)
modelo_forecast_C0.tail()

In [ ]:
# métricas de erro
for err_name, err_value in calculate_forecast_errors(modelo_forecast_C0, len(test_C0)).items():
    print(err_name, err_value)

#### Previsão cluster 4

In [ ]:
# Venda semanal do cluster 4
# Usaremos a venda média por semana das lojas do cluster 4

cluster_quatro = ['1','11','2','31','32','39','41','6','8']

def listaFiltro(dataframe, valores):
    return dataframe.loc[dataframe['loja'].isin(valores)]


vendas_C4 = listaFiltro(vendas, cluster_quatro)
vendas_C4


print(vendas_C4.data.min())
print(vendas_C4.data.max())

aggregations_C4 = {'venda_semanal': 'mean'}

venda_total_C4 = vendas_C4.groupby('data').agg(aggregations_C4)


venda_total_C4.head()

In [ ]:
# define frequência semanal
venda_semanal_C4 = venda_total_C4[['venda_semanal']].asfreq('W-FRI')
venda_semanal_C4.info()
venda_semanal_C4.head()

In [ ]:
# plot série temporal
venda_semanal_C4.plot(title='Venda semanal')

In [ ]:
# Train e test
train_C4 = venda_semanal_C4[:'2012-06-29']
test_C4 = venda_semanal_C4['2012-07-06':]

In [ ]:
# Decomposição
decomposition_C4 = seasonal_decompose(train_C4['venda_semanal'], model='multiplicative')  
# teste tambem com model='additive' e veja se você nota alguma diferença

# Outra forma possível de plotar a decomposição
plt.rc("figure",figsize=(20,10))
decomposition_C4.plot()
plt.show()

In [ ]:
# Buscando o modelo com a melhor performance
arima_model_C4 = auto_arima(train_C4, start_p=1, start_q=2,
                           max_p=5, max_q=5, m=52,
                           start_P=1, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',
                           suppress_warnings=True, 
                           stepwise=True)
arima_model_C4.summary()

In [ ]:
#n_periods = 17 até 26/10/2012
prediction_C4 = pd.DataFrame(arima_model_C4.predict(n_periods = 17))
prediction_C4.columns = ['predicted_sales']                       

#n_periods = 43 até 26/04/2013
prediction_6meses_C4 = pd.DataFrame(arima_model_C4.predict(n_periods = 43))
prediction_6meses_C4.columns = ['predicted_sales']  

In [ ]:
print(prediction_C4.tail())
print(prediction_6meses_C4.tail())

In [ ]:
# plot results - C0
plt.rc("figure",figsize=(15,5))
plt.plot(test_C4['venda_semanal'], color='orange', label= 'teste')
plt.plot(prediction_C4, color='red', label = 'previsão')
plt.legend()
plt.xticks(rotation = 45)
plt.show()

In [ ]:
#plot results - C0 - 6 meses
plt.plot(test_C4['venda_semanal'], color='orange', label= 'teste')
plt.plot(prediction_6meses_C4, color='red', label = 'previsão')
plt.legend()
plt.xticks(rotation = 45)
plt.show()

In [ ]:
# dataset para medir erro
modelo_forecast_C4 = pd.concat([train_C4.append(test_C4),prediction_C4['predicted_sales'].rename('previsao')], axis=1, sort=False)
modelo_forecast_C4.tail()

In [ ]:
# métricas de erro
for err_name, err_value in calculate_forecast_errors(modelo_forecast_C4, len(test_C4)).items():
    print(err_name, err_value)